# Convex Programming Example 2

This notebook uses convex programming to solve the water allocation problem from Chapter 4 Section 3.3.1 of Loucks, D. P., & van Beek, E. (2017). *Water resources planning and management: An introduction to methods, models, and applications*. $x$ represents the amount of water to allocate to 3 firms and $p$ the amount of product they produce with that water.

In [ ]:
!pip install cplex

In [ ]:
import numpy as np
import cplex
import cvxpy as cvx

for AW in np.arange(10,41,10):
    # Create vector variables of the allocations and prices
    x1 = cvx.Variable()
    x2 = cvx.Variable()
    x3 = cvx.Variable()
    p1 = cvx.Variable()
    p2 = cvx.Variable()
    p3 = cvx.Variable()

    B1 = 12*p1 - p1**2
    B2 = 20*p2 - 1.5*p2**2
    B3 = 28*p3 - 2.5*p3**2

    C1 = 3*p1**1.3
    C2 = 5*p2**1.2
    C3 = 6*p3**1.15

    obj = cvx.Maximize(B1 + B2 + B3 - C1 - C2 - C3)

    constraints = [0.4*x1**0.9 >= p1,
                   0.5*x2*0.8 >= p2,
                   0.6*x3**0.7 >= p3,
                   x1 + x2 + x3 <= AW,
                   x1 >= 0,
                   x2 >= 0,
                   x3 >= 0,
                   p1 >= 0,
                   p2 >= 0,
                   p3 >= 0]

    prob = cvx.Problem(obj, constraints)
    prob.solve(solver=cvx.CPLEX)

    print('AW = ', AW)
    print('Objective = $%0.2f' % obj.value)
    print('X1 = %0.2f' % x1.value)
    print('X2 = %0.2f' % x2.value)
    print('X3 = %0.2f' % x3.value)
    print('P1 = %0.2f' % p1.value)
    print('P2 = %0.2f' % p2.value)
    print('P3 = %0.2f' % p3.value)
    for i, constraint in enumerate(constraints):
        print('Constraint %(constraint)i Shadow Price = %(dual)0.2f' % {"constraint": i+1, "dual": constraint.dual_value})
    print('')

Make a 3 x 1 panel figure showing:  
1) the allocations to each user vs. available water,  
2) the price to each user vs. available water, and  
3) the net benedits vs. available water.

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

# re-run optimization across different values of AW and store solution values in vectors
x1s, x2s, x3s, p1s, p2s, p3s, objs = [[] for _ in range(7)]
AWs = np.arange(10,41,10)
for AW in AWs:
    x1 = cvx.Variable()
    x2 = cvx.Variable()
    x3 = cvx.Variable()

    p1 = cvx.Variable()
    p2 = cvx.Variable()
    p3 = cvx.Variable()

    B1 = 12*p1 - p1**2
    B2 = 20*p2 - 1.5*p2**2
    B3 = 28*p3 - 2.5*p3**2

    C1 = 3*p1**1.3
    C2 = 5*p2**1.2
    C3 = 6*p3**1.15

    obj = cvx.Maximize(B1 + B2 + B3 - C1 - C2 - C3)

    constraints = [0.4*x1**0.9 >= p1,
                   0.5*x2**0.8 >= p2,
                   0.6*x3**0.7 >= p3,
                   x1 + x2 + x3 <= AW,
                   x1 >= 0,
                   x2 >= 0,
                   x3 >= 0,
                   p1 >= 0,
                   p2 >= 0,
                   p3 >= 0]

    prob = cvx.Problem(obj, constraints)
    prob.solve(solver=cvx.CPLEX)

    x1s.append(x1.value)
    x2s.append(x2.value)
    x3s.append(x3.value)
    p1s.append(p1.value)
    p2s.append(p2.value)
    p3s.append(p3.value)
    objs.append(obj.value)

# plot solution values vs. AW
fig = plt.figure()

# allocations vs. AW
ax1 = fig.add_subplot(3,1,1)
ax1.plot(AWs,x1s,c="tab:blue",label="x1")
ax1.plot(AWs,x2s,c="tab:red",label="x2")
ax1.plot(AWs,x3s,c="tab:green",label="x3")
handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels)
ax1.set_ylabel("Water allocation")

# prices vs. AW
ax2 = fig.add_subplot(3,1,2)
ax2.plot(AWs,p1s,c="tab:blue",label="p1")
ax2.plot(AWs,p2s,c="tab:red",label="p2")
ax2.plot(AWs,p3s,c="tab:green",label="p3")
handles, labels = ax2.get_legend_handles_labels()
ax2.legend(handles, labels)
ax2.set_ylabel("Production")

# net benefits vs. AW
ax3 = fig.add_subplot(3,1,3)
ax3.plot(AWs,objs,c="black")
ax3.set_ylabel("Net Benefits")
ax3.set_xlabel("Available Water")

fig.tight_layout()
fig.show()

Repeat the above process using vector decision variables and parameters

In [ ]:
# AW must be nonnegative due to DCP rules.
AW = cvx.Parameter(nonneg=True)

# Make x and p vector decision variables and use linear algebra wherever possible
x = cvx.Variable(3)
p = cvx.Variable(3)

A1 = np.array([12, 20, 28])
A2 = np.array([1.0, 1.5, 2.5])
B = A1 @ p - A2 @ (p**2)

C = 3*p[0]**1.3 + 5*p[1]**1.2 + 6*p[2]**1.15

constraints = [0.4*x[0]**0.9 >= p[0],
               0.5*x[1]**0.8 >= p[1],
               0.6*x[2]**0.7 >= p[2],
               cvx.sum(x) <= AW,
               x >= 0,
               p >= 0]

obj = cvx.Maximize(B-C)
prob = cvx.Problem(obj, constraints)

# Solve for different values of AW
objs, x_values, p_values = [[] for _ in range(3)]
AW_vals = np.arange(10,41,10)
for AW_val in AW_vals:
    AW.value = AW_val
    prob.solve(solver=cvx.CPLEX)
    objs.append(obj.value)
    x_values.append(x.value)
    p_values.append(p.value)

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(311)
for i in range(3):
    ax1.plot(AW_vals, [xi[i] for xi in x_values],label='x' + str(i+1))

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels)
ax1.set_ylabel("Water allocation")

ax2 = fig.add_subplot(312)
for i in range(3):
    ax2.plot(AW_vals, [pi[i] for pi in p_values],label='p' + str(i+1))

handles, labels = ax2.get_legend_handles_labels()
ax2.legend(handles, labels)
ax2.set_ylabel("Production")

ax3 = fig.add_subplot(313)
ax3.plot(AW_vals, objs)
ax3.set_ylabel("Net Benefits")
ax3.set_xlabel("Available Water")

fig.tight_layout()
fig.show()